In [71]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score, 
                             accuracy_score, roc_auc_score, balanced_accuracy_score, f1_score)
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import pairwise_distances

from tqdm.notebook import tqdm
import time

In [20]:
DEBUG = False
print_limits = {"tokens" : 10}
prints = {"tokens": 0}
random_seed = 42

# Preprocessing

In [3]:
def tokenize_smiles(smiles):
    """
    Tokenizes a SMILES string into chemically meaningful units.
        
    Returns:
        list: A list of string tokens.
    """
    # This pattern prioritizes:
    # 1. Bracketed atoms (e.g., [nH], [Na+]) -> catch complex states first
    # 2. Two-letter atoms (Br, Cl) -> catch before single letters match
    # 3. Single-letter atoms (B, C, N, O...)
    # 4. Special characters (bonds, parens, digits)
    pattern =  r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|H|B|C|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
    
    regex = re.compile(pattern)
    tokens = [token for token in regex.findall(smiles)]

    if DEBUG and prints["tokens"] < print_limits["tokens"]: 
        print(f"[Tokenizer] Input: {smiles} -> Tokens: {tokens}")
        prints["tokens"] += 1

    return tokens

In [18]:
class KSpectrumVectorizer:
    def __init__(self, max_k=3):
        self.max_k = max_k
        self.vectorizer = None
        self.feature_names = None
        self.k_masks = {} # Stores which columns belong to k=1, k=2, etc.

    def fit_transform(self, smiles_list):
        if DEBUG:
            print(f"\n--- Starting Vectorization (Max k={self.max_k}) ---")
        
        self.vectorizer = CountVectorizer(
            tokenizer=tokenize_smiles,
            token_pattern=None, 
            ngram_range=(1, self.max_k),
            lowercase=False,
            dtype=np.float32
        )
        
        X_sparse = self.vectorizer.fit_transform(smiles_list)
        
        self.feature_names = self.vectorizer.get_feature_names_out()
        
        if DEBUG:
            print(f"[Vectorizer] Total unique substrings found: {len(self.feature_names)}")
            print(f"[Vectorizer] First 10 features: {self.feature_names[:10]}")
            print("[Vectorizer] Creating k-masks for distance metric...")
        
        k_counts = np.array([name.count(' ') + 1 for name in self.feature_names])
        
        for k in range(1, self.max_k + 1):
            # Create a boolean array: True if column belongs to k
            self.k_masks[k] = (k_counts == k)
            
            if DEBUG:
                count = np.sum(self.k_masks[k])
                print(f"  > Found {count} unique substrings of length k={k}")

        return X_sparse

In [5]:
def make_distance_function(k_masks):
    def dist_func(x_vec, y_vec):
        if hasattr(x_vec, "toarray"): x_vec = x_vec.toarray().flatten()
        if hasattr(y_vec, "toarray"): y_vec = y_vec.toarray().flatten()
        
        diff = np.abs(x_vec - y_vec)
        total_sq_dist = 0.0
        
        for k, mask in k_masks.items():
            d_k = np.sum(diff[mask])
            total_sq_dist += d_k ** 2
            
        return np.sqrt(total_sq_dist)
    return dist_func

# Experiments

In [62]:
def run_experiment(experiment_name, model, param_grid, X_train, X_test, y_train, y_test, mode='regression'):
    """
    Runs a GridSearch experiment, evaluates on test set, and updates the global results dataframe.
    """
    global df_regression, df_classification
    
    print(f"--- Running Experiment: {experiment_name} ({mode}) ---")
    start_time = time.time()
    
    # 1. Grid Search to find best parameters
    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,
        scoring='neg_mean_squared_error' if mode == 'regression' else 'accuracy',
        n_jobs=-1
    )
    
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    best_params = grid.best_params_
    
    # 2. Prediction on the test set
    y_pred = best_model.predict(X_test)
    
    # 3. Calculate metrics
    elapsed_time = time.time() - start_time
    
    if mode == 'regression':
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        new_row = pd.DataFrame([{
            'experiment': experiment_name,
            'best_params': str(best_params),
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'time_s': elapsed_time
        }])
        if not new_row.empty:
             df_regression = pd.concat([df_regression, new_row], ignore_index=True)
        
        print(f"  > Best Params: {best_params}")
        print(f"  > RMSE: {rmse:.4f} | R2: {r2:.4f}")

    elif mode == 'classification':
        acc = accuracy_score(y_test, y_pred)
        bal_acc = balanced_accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted') 
        
        try:
            if hasattr(best_model, "predict_proba"):
                if len(np.unique(y_train)) == 2:
                    y_prob = best_model.predict_proba(X_test)[:, 1]
                else:
                    y_prob = best_model.predict_proba(X_test) # Multiclass
                roc = roc_auc_score(y_test, y_prob, multi_class='ovr')
            else:
                roc = np.nan
        except Exception as e:
            print(f"  ! Could not calc ROC-AUC: {e}")
            roc = np.nan

        new_row = pd.DataFrame([{
            'experiment': experiment_name,
            'best_params': str(best_params),
            'accuracy': acc,
            'balanced_accuracy': bal_acc,
            'f1': f1,
            'roc_auc': roc,
            'time_s': elapsed_time
        }])
        
        if not new_row.empty:
            df_classification = pd.concat([df_classification, new_row], ignore_index=True)
        
        print(f"  > Best Params: {best_params}")
        print(f"  > Acc: {acc:.4f} | ROC-AUC: {roc:.4f}")

    print(f"  > Time: {elapsed_time:.2f}s\n")

In [47]:
def style_results(df, mode='regression'):
    """
    Applies green (best) and red (worst) highlighting to a results dataframe.
    """
    styled = df.copy().style.format(precision=4)
    
    best_style = 'background-color: #d4edda; color: #155724; font-weight: bold;' # Green
    worst_style = 'background-color: #f8d7da; color: #721c24;' # Red
    
    def highlight_col(s, higher_is_better=True):
        is_best = s == s.max() if higher_is_better else s == s.min()
        is_worst = s == s.min() if higher_is_better else s == s.max()
        
        return [best_style if b else worst_style if w else '' 
                for b, w in zip(is_best, is_worst)]

    # Apply based on mode
    if mode == 'regression':
        styled = styled.apply(highlight_col, subset=['mae', 'rmse'], higher_is_better=False)
        styled = styled.apply(highlight_col, subset=['r2'], higher_is_better=True)
        
    elif mode == 'classification':
        metrics = ['accuracy', 'balanced_accuracy', 'f1', 'roc_auc']
        existing_metrics = [m for m in metrics if m in df.columns]
        
        styled = styled.apply(highlight_col, subset=existing_metrics, higher_is_better=True)

    return styled

In [63]:
df_regression = pd.DataFrame(columns=[
    'experiment', 'best_params', 'mae', 'rmse', 'r2', 'time_s'
])
df_classification = pd.DataFrame(columns=[
    'experiment', 'best_params', 'accuracy', 'balanced_accuracy', 'f1', 'roc_auc', 'time_s'
])

## ESOL

In [41]:
df = pd.read_csv('esol.csv')
col_smiles = 'smiles'
col_target = 'measured log solubility in mols per litre'

### kNN

In [64]:
largest_k = 5
print(f"Pre-calculating features for max_k={largest_k}...")

# Initialize vectorizer with the absolute maximum k
global_vectorizer = KSpectrumVectorizer(max_k=largest_k)
X_all = global_vectorizer.fit_transform(df[col_smiles].astype(str).tolist())
y_all = df[col_target].values
global_masks = global_vectorizer.k_masks

# Train/test split fixed for all experiments
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.3, random_state=random_seed
)

k_values_to_test = [1, 2, 3, 4, 5]

for k in k_values_to_test:
    
    # Create a subset of masks that only contains keys 1..k
    subset_masks = {i: global_masks[i] for i in range(1, k + 1)}
    
    # Create a metric that only sees the columns relevant to this k
    current_metric = make_distance_function(subset_masks)
    
    knn_custom = KNeighborsRegressor(
        metric=current_metric, 
        algorithm='brute',
        n_jobs=-1
    )
    
    params_knn = {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance']
    }
    
    # Run using the shared X_train/X_test
    # Note: X_train has columns for k > current k_max, but the metric will ignore them 

    run_experiment(
        f"KNN (max_k={k})", 
        knn_custom, 
        params_knn, 
        X_train, X_test, y_train, y_test, 
        mode='regression'
    )

Pre-calculating features for max_k=5...
--- Running Experiment: KNN (max_k=1) (regression) ---


C:\Users\Baaa\AppData\Local\Temp\ipykernel_14760\3153140418.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_regression = pd.concat([df_regression, new_row], ignore_index=True)


  > Best Params: {'n_neighbors': 5, 'weights': 'distance'}
  > RMSE: 0.8092 | R2: 0.8565
  > Time: 78.74s

--- Running Experiment: KNN (max_k=2) (regression) ---
  > Best Params: {'n_neighbors': 5, 'weights': 'distance'}
  > RMSE: 0.8587 | R2: 0.8383
  > Time: 82.34s

--- Running Experiment: KNN (max_k=3) (regression) ---
  > Best Params: {'n_neighbors': 5, 'weights': 'distance'}
  > RMSE: 0.9534 | R2: 0.8007
  > Time: 91.31s

--- Running Experiment: KNN (max_k=4) (regression) ---
  > Best Params: {'n_neighbors': 3, 'weights': 'distance'}
  > RMSE: 1.0772 | R2: 0.7456
  > Time: 110.43s

--- Running Experiment: KNN (max_k=5) (regression) ---
  > Best Params: {'n_neighbors': 3, 'weights': 'distance'}
  > RMSE: 1.2188 | R2: 0.6743
  > Time: 122.39s



### Random Forest

In [68]:
largest_k = 5
print(f"Pre-calculating features for max_k={largest_k}...")

# Initialize vectorizer with the absolute maximum k
global_vectorizer = KSpectrumVectorizer(max_k=largest_k)
X_all = global_vectorizer.fit_transform(df[col_smiles].astype(str).tolist())
y_all = df[col_target].values

X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.3, random_state=random_seed
)

rf_model = RandomForestRegressor(random_state=random_seed, n_jobs=-1)

params_rf = {
    'n_estimators': [50, 100, 150, 200, 250, 300],      # Number of trees
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 2, 4]
}

run_experiment(
    "Random Forest (All Features)", 
    rf_model, 
    params_rf, 
    X_train, X_test, y_train, y_test, 
    mode='regression'
)

Pre-calculating features for max_k=5...
--- Running Experiment: Random Forest (All Features) (regression) ---
  > Best Params: {'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 200}
  > RMSE: 0.8357 | R2: 0.8469
  > Time: 135.60s



### Kernel Ridge Regression

In [73]:
# Wrapper class, scikit does not defaultly support custom distance functions in Kernel Ridge Regression
class KSpectrumKRR(BaseEstimator, RegressorMixin):
    def __init__(self, k_masks=None, alpha=1.0, gamma=0.1):
        self.k_masks = k_masks
        self.alpha = alpha  # Regularization strength
        self.gamma = gamma  # Kernel width (1/sigma^2)
        self.model = None
        self.X_train_ = None

    def fit(self, X, y):
        self.X_train_ = X
        
        # 1. Create the distance function
        dist_func = make_distance_function(self.k_masks)
        
        # 2. Compute distance matrix (Train vs Train)
        D = pairwise_distances(X, metric=dist_func)
        
        # 3. Convert distance to kernel
        # K = exp(-gamma * distance^2)
        K = np.exp(-self.gamma * D**2)
        
        # 4. Fit standard kernel ridge with precomputed kernel
        self.model = KernelRidge(alpha=self.alpha, kernel='precomputed')
        self.model.fit(K, y)
        return self

    def predict(self, X):
        dist_func = make_distance_function(self.k_masks)
        
        D_test = pairwise_distances(X, self.X_train_, metric=dist_func)
        
        K_test = np.exp(-self.gamma * D_test**2)
        
        return self.model.predict(K_test)


largest_k = 5
print(f"Pre-calculating features for max_k={largest_k}...")

global_vectorizer = KSpectrumVectorizer(max_k=largest_k)
X_all = global_vectorizer.fit_transform(df[col_smiles].astype(str).tolist())
y_all = df[col_target].values
global_masks = global_vectorizer.k_masks

X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.3, random_state=random_seed
)

k_values_to_test = [1, 2, 3, 4, 5]

for k in k_values_to_test:
    
    subset_masks = {i: global_masks[i] for i in range(1, k + 1)}
    
    krr_custom = KSpectrumKRR(k_masks=subset_masks)
    
    # Parameter Grid
    # Alpha: Higher = more regularization (prevents overfitting)
    # Gamma: Higher = similarity drops off faster (only very close molecules matter)
    params_krr = {
        'alpha': [1e-3, 0.1, 1.0, 10],
        'gamma': [0.001, 0.01, 0.1, 0.5] 
    }
    
    run_experiment(
        f"KRR (max_k={k})", 
        krr_custom, 
        params_krr, 
        X_train, X_test, y_train, y_test, 
        mode='regression'
    )

Pre-calculating features for max_k=5...
--- Running Experiment: KRR (max_k=1) (regression) ---
  > Best Params: {'alpha': 10, 'gamma': 0.001}
  > RMSE: 1.6181 | R2: 0.4259
  > Time: 223.98s

--- Running Experiment: KRR (max_k=2) (regression) ---
  > Best Params: {'alpha': 10, 'gamma': 0.001}
  > RMSE: 1.8411 | R2: 0.2568
  > Time: 244.11s

--- Running Experiment: KRR (max_k=3) (regression) ---
  > Best Params: {'alpha': 10, 'gamma': 0.001}
  > RMSE: 2.4089 | R2: -0.2723
  > Time: 264.14s

--- Running Experiment: KRR (max_k=4) (regression) ---
  > Best Params: {'alpha': 10, 'gamma': 0.001}
  > RMSE: 2.8777 | R2: -0.8157
  > Time: 299.29s

--- Running Experiment: KRR (max_k=5) (regression) ---


c:\Users\Baaa\miniconda3\envs\ML\Lib\site-packages\sklearn\linear_model\_ridge.py:252: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


  > Best Params: {'alpha': 1.0, 'gamma': 0.001}
  > RMSE: 2.9188 | R2: -0.8679
  > Time: 338.91s



## Results

In [78]:
print("Regression Results:")
display(style_results(df_regression, mode='regression'))

Regression Results:


,experiment,best_params,mae,rmse,r2,time_s,dataset
0,KNN (max_k=1),"{'n_neighbors': 5, 'weights': 'distance'}",0.5991,0.8092,0.8565,78.7351,ESOL
1,KNN (max_k=2),"{'n_neighbors': 5, 'weights': 'distance'}",0.6300,0.8587,0.8383,82.3384,ESOL
2,KNN (max_k=3),"{'n_neighbors': 5, 'weights': 'distance'}",0.7074,0.9534,0.8007,91.3117,ESOL
3,KNN (max_k=4),"{'n_neighbors': 3, 'weights': 'distance'}",0.7717,1.0772,0.7456,110.4308,ESOL
4,KNN (max_k=5),"{'n_neighbors': 3, 'weights': 'distance'}",0.8582,1.2188,0.6743,122.3921,ESOL
5,Random Forest (All Features),"{'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 200}",0.6172,0.8357,0.8469,67.7005,ESOL
6,Random Forest (All Features),"{'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 200}",0.6172,0.8357,0.8469,135.6017,ESOL
7,KRR (max_k=1),"{'alpha': 1.0, 'gamma': 0.1}",1.6273,2.3566,-0.2176,146.2699,ESOL
8,KRR (max_k=1),"{'alpha': 10, 'gamma': 0.001}",1.2477,1.6181,0.4259,223.9843,ESOL
9,KRR (max_k=2),"{'alpha': 10, 'gamma': 0.001}",1.3820,1.8411,0.2568,244.1093,ESOL


In [77]:
df_regression["dataset"] = "ESOL"

In [79]:
df_regression.to_csv('regression_results.csv', index=False)

## BACE